### Testing a GPT Output of GAN

In [4]:
import os
import numpy as np
from PIL import Image
from sklearn.utils import shuffle
import tensorflow as tf


In [5]:
# check if a gpu is available
def check_gpu():
    if tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None):
        print("GPU is available")
    else:
        print("GPU is not available")

check_gpu()

GPU is not available


2024-05-01 04:59:09.997246: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
tf.config.list_physical_devices('GPU')

[]

In [4]:
import tensorflow as tf

# Define function to load and preprocess Pokémon dataset
def load_and_preprocess_dataset(dataset_path, image_shape, batch_size):
    # Create a dataset from image files
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        dataset_path,
        image_size=(image_shape[0], image_shape[1]),
        batch_size=batch_size
    )
    # Normalize pixel values to range [-1, 1]
    dataset = dataset.map(lambda x, _: (x / 127.5) - 1)
    return dataset

2024-04-30 14:47:40.557019: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 14:47:41.559155: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:


# Define dataset path, image shape, and batch size
dataset_path = 'path/to/pokemon/dataset'
image_shape = (64, 64)  # Adjust this according to your dataset
batch_size = 32  # Adjust batch size according to your system memory

# Load and preprocess Pokémon dataset
pokemon_dataset = load_and_preprocess_dataset(dataset_path, image_shape, batch_size)

# Print dataset shape
for images, _ in pokemon_dataset.take(1):
    print("Pokémon dataset shape:", images.shape)


In [ ]:



# Define function to load Pokémon dataset
def load_pokemon_dataset(dataset_path, image_shape):
    images = []
    # Iterate over each Pokémon image file
    for filename in os.listdir(dataset_path):
        if filename.endswith('.png'):  # Assuming images are in PNG format
            image = Image.open(os.path.join(dataset_path, filename))
            # Resize the image to the desired shape
            image = image.resize(image_shape[:2])
            # Convert image to numpy array
            image = np.array(image)
            # Normalize pixel values to range [-1, 1]
            image = (image.astype(np.float32) - 127.5) / 127.5
            # Add image to the list
            images.append(image)
    return np.array(images)


In [ ]:

# Define dataset path and image shape
dataset_path = 'path/to/pokemon/dataset'
image_shape = (64, 64, 3)  # Adjust this according to your dataset

# Load Pokémon dataset
pokemon_data = load_pokemon_dataset(dataset_path, image_shape)

# Shuffle the dataset
pokemon_data = shuffle(pokemon_data)

# Print dataset shape
print("Pokémon dataset shape:", pokemon_data.shape)


In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

# Load Pokémon dataset or any other dataset of your choice
# For simplicity, let's assume we're using the MNIST dataset for now
# You can replace this with your Pokémon dataset
# (Make sure your dataset is preprocessed and resized to a suitable resolution)
#(x_train, _), (_, _) = mnist.load_data()


In [ ]:

# Define the generator model
def build_generator(latent_dim, image_shape):
    model = keras.Sequential([
        layers.Dense(128, input_dim=latent_dim),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(256),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(512),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(np.prod(image_shape), activation='tanh'),
        layers.Reshape(image_shape)
    ])
    return model

# Define the discriminator model
def build_discriminator(image_shape):
    model = keras.Sequential([
        layers.Flatten(input_shape=image_shape),
        layers.Dense(512),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(256),
        layers.LeakyReLU(alpha=0.2),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# Define GAN model (combined generator and discriminator)
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = keras.Sequential([
        generator,
        discriminator
    ])
    return model

# Define parameters
latent_dim = 100
image_shape = (28, 28, 1) # Adjust this to match your image size

# Build and compile discriminator
discriminator = build_discriminator(image_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])

# Build generator
generator = build_generator(latent_dim, image_shape)

# Build and compile GAN model
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.0002, beta_1=0.5))

# Function to generate fake images
def generate_fake_samples(generator, latent_dim, n_samples):
    x_input = np.random.randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    X = generator.predict(x_input)
    y = np.zeros((n_samples, 1))
    return X, y

# Training loop
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, batch_size=128):
    batch_per_epoch = dataset.shape[0] // batch_size
    half_batch = batch_size // 2
    for epoch in range(n_epochs):
        for batch in range(batch_per_epoch):
            # Train discriminator
            x_real, y_real = dataset[np.random.randint(0, dataset.shape[0], half_batch)], np.ones((half_batch, 1))
            d_loss_real = d_model.train_on_batch(x_real, y_real)
            x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            d_loss_fake = d_model.train_on_batch(x_fake, y_fake)
            # Train generator
            x_gan = np.random.randn(latent_dim * batch_size)
            x_gan = x_gan.reshape(batch_size, latent_dim)
            y_gan = np.ones((batch_size, 1))
            g_loss = gan_model.train_on_batch(x_gan, y_gan)
            # Summarize progress
            print(f'Epoch {epoch+1}/{n_epochs}, Batch {batch}/{batch_per_epoch}, D_loss_real={d_loss_real[0]}, D_loss_fake={d_loss_fake[0]}, G_loss={g_loss}')
        # Visualize generated images
        if (epoch+1) % 10 == 0:
            visualize_generated_images(generator, epoch+1)

# Function to visualize generated images
def visualize_generated_images(generator, epoch, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, size=[examples, latent_dim])
    generated_images = generator.predict(noise)
    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(f'gan_generated_image_epoch_{epoch}.png')
    plt.show()

# Train GAN
#train(generator, discriminator, gan, x_train, latent_dim)
